<a href="https://colab.research.google.com/github/roaring60s/CYP2D6_CNV_Analysis/blob/main/CYP2D6_Exon_Coordinate_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title A Bioinformatic Protocol for the Definitive Identification of Human *CYP2D6* Exon Coordinates on the GRCh38 Reference Assembly
# @markdown # A Bioinformatic Protocol for the Definitive Identification of Human *CYP2D6* Exon Coordinates on the GRCh38 Reference Assembly
# @markdown ---
# @markdown ## 1. Introduction: The *CYP2D6* Gene Locus - A Nexus of Pharmacogenomic Complexity
# @markdown The human cytochrome P450 (CYP) superfamily of enzymes represents a cornerstone of xenobiotic metabolism, and among its members, Cytochrome P450 2D6 (CYP2D6) holds a position of exceptional clinical significance. Encoded by the *CYP2D6* gene, this enzyme is responsible for the oxidative metabolism of a vast and diverse array of clinically administered drugs. It is estimated that CYP2D6 metabolizes as many as 25% of all commonly prescribed medications, placing it at the center of modern pharmacogenomic inquiry.
# @markdown
# @markdown The profound clinical relevance of *CYP2D6* is magnified by its extreme genetic variability. The gene is highly polymorphic, with hundreds of documented allelic variants that result in a wide spectrum of enzymatic activity, leading to distinct pharmacogenomic phenotypes (e.g., poor, intermediate, extensive, and ultrarapid metabolizers).
# @markdown
# @markdown This clinical imperative, however, is confronted by a formidable genomic challenge. The *CYP2D6* gene, mapped to chromosome 22q13.2, resides in a complex locus complicated by the presence of a highly homologous but non-functional pseudogene, *CYP2D7*. This high degree of sequence identity is a notorious source of error in molecular assays and bioinformatic analyses.
# @markdown
# @markdown Consequently, this protocol's objective is to identify the exon coordinates for the canonical **reference allele** of *CYP2D6* as represented on the GRCh38 primary reference assembly. This reference sequence serves as the fundamental coordinate system upon which all other allelic variations are described.
# @markdown ---
# @markdown ## 2. A Programmatic Approach: Interfacing with NCBI via E-utilities
# @markdown To implement the data retrieval strategy, we will interface directly with NCBI's backend databases using the Entrez Programming Utilities (E-utilities). The workflow is as follows:
# @markdown
# @markdown 1.  **Dynamically Discover Gene Location:** Use the stable NCBI Gene ID for *CYP2D6* (1565) to programmatically query the NCBI Gene database and find its exact chromosomal location on the GRCh38 assembly.
# @markdown 2.  **Fetch Annotated Genomic Slice:** Request only the specific ~4.3 kb slice of the chromosome containing the gene. This is highly efficient.
# @markdown 3.  **Parse and Extract:** Parse the resulting GenBank-formatted data to extract all features annotated as 'exon' that explicitly belong to the *CYP2D6* gene. This provides the final coordinates.
# @markdown ---

# @title 3. Implementation: The Complete Python Workflow
# @markdown This section contains the complete, runnable Python code to execute the analysis.
# @markdown
# @markdown ### Step 3.1: Install Prerequisites
# @markdown The script requires the Biopython library. The following cell will install it.

# In[1]:
!pip install biopython

# @markdown ### Step 3.2: The Main Python Script
# @markdown The following cell contains the full script. It will:
# @markdown 1. Connect to NCBI.
# @markdown 2. Dynamically find the location of *CYP2D6* on chromosome 22.
# @markdown 3. Fetch the annotated genomic region.
# @markdown 4. Parse the data and filter for *CYP2D6* exons.
# @markdown 5. Print the final coordinates in a formatted table.
# @markdown
# @markdown **Note:** You must replace `"your.email@example.com"` with your actual email address to comply with NCBI's usage guidelines.

# In[2]:
import sys
from Bio import Entrez
from Bio import SeqIO
from xml.etree import ElementTree

def get_cyp2d6_exon_coordinates_from_ncbi():
    """
    Main function to fetch, parse, and display CYP2D6 exon coordinates from NCBI.
    """
    # --------------------------------------------------------------------------
    # E-utilities Configuration
    # Always tell NCBI who you are. This is a requirement.
    # --------------------------------------------------------------------------
    Entrez.email = "your.email@example.com"  # <<< IMPORTANT: REPLACE WITH YOUR EMAIL
    Entrez.tool = "CYP2D6_Exon_Finder_Colab_v1.0"

    gene_id = "1565"  # NCBI Gene ID for human CYP2D6
    assembly_name = "GRCh38" # Target assembly for coordinate extraction

    print(f"--- Starting analysis for CYP2D6 (Gene ID: {gene_id}) on {assembly_name} ---")

    try:
        # --------------------------------------------------------------------------
        # Dynamic Retrieval of Gene Location
        # --------------------------------------------------------------------------
        print("\n1. Fetching gene summary from NCBI to find genomic location...")
        handle = Entrez.efetch(db="gene", id=gene_id, rettype="docsum", retmode="xml")
        xml_data = handle.read()
        handle.close()

        root = ElementTree.fromstring(xml_data)
        location_data = None

        # Find the primary chromosomal location from the XML summary
        genomic_infos = root.findall('.//GenomicInfoType')
        for info in genomic_infos:
            accession = info.find('ChrAccVer')
            start = info.find('ChrStart')
            stop = info.find('ChrStop')

            # Look for the primary assembly placement, which starts with 'NC_'
            if accession is not None and accession.text.startswith("NC_"):
                chrom_accession = accession.text
                chrom_start = int(start.text)
                chrom_stop = int(stop.text)
                location_data = (chrom_accession, chrom_start, chrom_stop)
                print(f"   > Found primary location: {chrom_accession}, Start: {chrom_start}, Stop: {chrom_stop}")
                break

        if not location_data:
            raise ValueError(f"Could not find location data for assembly {assembly_name}.")

        chrom_accession, chrom_start, chrom_stop = location_data

        # --------------------------------------------------------------------------
        # Fetching the Annotated Genomic Slice
        # --------------------------------------------------------------------------
        print("\n2. Fetching annotated genomic slice from Nuccore database...")
        handle = Entrez.efetch(
            db="nuccore",
            id=chrom_accession,
            rettype="gb",
            retmode="text",
            seq_start=chrom_start,
            seq_stop=chrom_stop,
        )

        # --------------------------------------------------------------------------
        # Parsing with Bio.SeqIO and Feature Extraction
        # --------------------------------------------------------------------------
        print("\n3. Parsing GenBank record and extracting exon features...")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        exons = []
        # In this complex locus, we find the 'gene' feature first to set the context,
        # then collect all subsequent 'exon' features until we hit the next gene.
        in_cyp2d6_gene = False
        for feature in record.features:
            if feature.type == "gene":
                if 'gene' in feature.qualifiers and feature.qualifiers['gene'][0] == 'CYP2D6':
                    in_cyp2d6_gene = True
                else:
                    # If we encounter another gene, stop collecting exons for CYP2D6
                    in_cyp2d6_gene = False

            # If we are in the CYP2D6 gene context, collect the exons
            if feature.type == "exon" and in_cyp2d6_gene:
                exon_number_list = feature.qualifiers.get('number', ['N/A'])
                exon_number = exon_number_list[0] if exon_number_list else 'N/A'

                start_coord = feature.location.start + 1
                end_coord = feature.location.end
                strand = feature.location.strand
                length = len(feature)

                exons.append({
                    "number": int(exon_number),
                    "chromosome": record.name,
                    "start": start_coord,
                    "end": end_coord,
                    "strand": strand,
                    "length": length
                })

        if not exons:
            print("   > No exons for CYP2D6 found in the fetched record. Check annotations.")
            return

        exons.sort(key=lambda x: x['number'])

        # --------------------------------------------------------------------------
        # Final Results Presentation
        # --------------------------------------------------------------------------
        print("\n--- Final Results: CYP2D6 Exon Coordinates on GRCh38 ---")
        print("-" * 80)
        header = f"{'Exon':<6} | {'Chromosome':<15} | {'Start (1-based)':<18} |